# Criar VENV

https://oandersonbm.medium.com/usando-python-no-vs-code-com-ambiente-virtual-venv-ecef7959b652

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import locale

# Definindo a configuração local para o Brasil (pt_BR)
locale.setlocale(locale.LC_ALL, 'pt_BR')

# Personalizando para ver todas as colunas do DataFrame
pd.set_option('display.max_columns', None)

# Base de dados

In [2]:
# Realizando a junção dos 3 arquivos .CSV num único DataFrame

# Caminhos para os arquivos .CSV
caminho_1 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini.csv"

caminho_2 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(1).csv"

caminho_3 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(2).csv"

caminho_4 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(3).csv"


# Leitura dos arquivos .CSV em DataFrames individuais
df1 = pd.read_csv(caminho_1)
df2 = pd.read_csv(caminho_2)
df3 = pd.read_csv(caminho_3)
df4 = pd.read_csv(caminho_4)

# Concatenação dos DataFrames em um único DataFrame
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Excluindo linhas que contenham a string "Tela" em qualquer coluna
df = df[~df.apply(lambda row: row.astype(str).str.contains('Tela')).any(axis=1)]

# Transformando o tipo de dado das colunas de valores monetários
df['A_LIQUIDAR'] = df['A_LIQUIDAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['LIQUIDADO_A_PAGAR'] = df['LIQUIDADO_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['TOTAL_A_PAGAR'] = df['TOTAL_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['PAGO'] = df['PAGO'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

# Convertendo a coluna DATA para datetime
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%y')

# Convertendo a coluna DIAS em número inteiro
df['DIAS'] = df['DIAS'].astype(int)



# Trabalhando com um DATAFRAME voltado para controle dos SALDOS

In [3]:
df_saldos = df[['SIGLA UG', 'ANO', 'NOME_CREDOR', 'DATA', 'DIAS', 'ND','NDSI', 'NOME_NDSI', 'NE', 'PI', 'NOME_PI','TIPO', 'NOME_UGR', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']]

In [4]:
# Saldos Totais
total_pago = df['PAGO'].sum().round(2)
total_a_liquidar = df['A_LIQUIDAR'].sum().round(2)
total_liquidado_a_pagar = df['LIQUIDADO_A_PAGAR'].sum().round(2)
total_a_pagar = df['TOTAL_A_PAGAR'].sum().round(2)


print(f'O total pago pela ESA até o momento foi: {locale.currency(total_pago, grouping=True)}')
print(f'O total de saldo a liquidar pela ESA até o momento foi: {locale.currency(total_a_liquidar, grouping=True)}')
print(f'O total de saldo liquidado a pagar pela ESA até o momento foi: {locale.currency(total_liquidado_a_pagar, grouping=True)}')
print(f'O total de saldo a pagar pela ESA até o momento foi: {locale.currency(total_a_pagar, grouping=True)}')

O total pago pela ESA até o momento foi: R$ 26.447.002,72
O total de saldo a liquidar pela ESA até o momento foi: R$ 6.228.111,09
O total de saldo liquidado a pagar pela ESA até o momento foi: R$ 393.153,53
O total de saldo a pagar pela ESA até o momento foi: R$ 6.621.264,62


In [5]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ND', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ND',
             labels={'ND':'Natureza da Despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=500,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [6]:
# Saldo a liquidar por Subitem
cols = ['NDSI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NDSI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NDSI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por NDSI',
             labels={'NDSI':'Subitem da despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=600,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [7]:
# Saldo a liquidar por tipo de PI
cols = ['NOME_PI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_PI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_PI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por PI',
             labels={'NOME_PI':'Tipo de PI', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=800,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [8]:
# Saldo a liquidar por tipo de Empenho
cols = ['TIPO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por tipo de empenho',
             labels={'TIPO':'Tipo de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [9]:
# Saldo a liquidar por ano
cols = ['ANO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ANO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ANO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'ANO':'Ano do Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [10]:
# Saldo a liquidar por tipo de NE
cols = ['TIPO_NE', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO_NE').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO_NE', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'TIPO_NE':'Tipo de Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [11]:
# Saldo a liquidar por tipo de NE
cols = ['NOME_UGR', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_UGR').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_UGR', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'NOME_UGR':'Nome das UGR', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [12]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'PAGO']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
# Criando o gráfico
fig = go.Figure()

# Adicionando as colunas y1, y2 e y3 no eixo y
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['A_LIQUIDAR'], name='A Liquidar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['LIQUIDADO_A_PAGAR'], name='Liquidado a pagar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['PAGO'], name='Pago'))

fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")
fig.update_yaxes(type="log")

fig.show()

# Suporte para o Dashboard

In [13]:
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']

df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
df_aux

,NE,A_LIQUIDAR,DIAS,NOME_NDSI,NDSI
1134,2022NE001087,2397.00,392,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,33904012
1168,2022NE001086,3466.70,392,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,33904012
1167,2022NE001085,5543.35,392,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,33903917
1157,2022NE002421,5758.75,337,SERVICOS GRAFICOS E EDITORIAIS,33903963
1419,2022NE001843,1016.25,337,SERVICOS GRAFICOS E EDITORIAIS,33903963
954,2022NE002483,263051.08,323,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
953,2022NE002482,155796.97,323,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
1154,2022NE002703,146.93,309,PASSAGENS PARA O PAIS,33903301
1150,2022NE002702,12733.76,309,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
1148,2022NE002700,487.75,309,PASSAGENS PARA O PAIS,33903301


In [14]:
# Saldo a liquidar por tipo de NE
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']
df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
fig = px.bar(df_aux, 
             x='NE', 
             y='A_LIQUIDAR',
             title='10 empenhos com mais tempo na tela',
             labels={'NE':'Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             color='DIAS',
             color_continuous_scale = 'reds',
             log_y=True,
             template='plotly_dark',
             hover_name="NE", hover_data={'DIAS': True, 'A_LIQUIDAR': True, 'NOME_NDSI': True, 'NDSI': True})
fig.update_traces(textposition = 'outside')


fig.show()

## Informações para SideBar

In [15]:
# Qtde de empenhos realizados até o momento
df.shape[0]

3263

# Requisição em API para consulta a CNPJ

In [16]:
import requests
from pprint import pprint

url = "https://receitaws.com.br/v1/cnpj/21331404000138"

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

pprint(response.json())

{'abertura': '03/11/2014',
 'atividade_principal': [{'code': '79.11-2-00', 'text': 'Agências de viagens'}],
 'atividades_secundarias': [{'code': '59.11-1-99',
                             'text': 'Atividades de produção cinematográfica, '
                                     'de vídeos e de programas de televisão '
                                     'não especificadas anteriormente'},
                            {'code': '63.99-2-00',
                             'text': 'Outras atividades de prestação de '
                                     'serviços de informação não especificadas '
                                     'anteriormente'},
                            {'code': '79.12-1-00',
                             'text': 'Operadores turísticos'},
                            {'code': '79.90-2-00',
                             'text': 'Serviços de reservas e outros serviços '
                                     'de turismo não especificados '
                                    

In [17]:
def consulta_cnpj(numero_cnpj):
    url = f"https://receitaws.com.br/v1/cnpj/{numero_cnpj}"

    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers)
    
    dic_fornecedor = response.json()
    
    return dic_fornecedor

# Criar um dicionário vazio fora da função
dicionario = {}

# Chamar a função e atribuir o resultado ao dicionário
dicionario = consulta_cnpj('81243735001977')

# Imprimir o dicionário
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [18]:
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [19]:
dicionario['nome']

'POSITIVO TECNOLOGIA S.A.'

In [20]:
dicionario['telefone']

'(41) 3316-7700'

In [21]:
dicionario['email']

'fiscal@positivo.com.br'

In [22]:
dicionario['logradouro'] + ', ' + dicionario['numero'] + ' - ' + dicionario['bairro'] + ' - ' + dicionario['municipio'] + ' / ' + dicionario['uf']

'R JAVARI, 1255 - DISTRITO INDUSTRIAL I - MANAUS / AM'

In [23]:
# filtrando dos dados da coluna FAV
df_cnpj = df[df['NOME_PI'] != 'PROFISSIONAL DE SAUDE AUTONOMO']
df_cnpj = df[df['NOME_PI'] != 'ATENDIMENTO MEDICO-HOPITALR/FC']
df_cnpj = df[df['NOME_PI'] != 'IND-INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'IND - INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'INDENIZACAO DESPESAS.']
df_cnpj = df[df['NOME_PI'] != 'AUXILIO FINANCEIRO NÃO INDENIZÁVEL']
df_cnpj = df[df['NOME_PI'] != 'INRE - RESTITUICAO E RESSARCIMENTO']
df_cnpj = df[df['NOME_NDSI'] != 'OUTROS SERV.DE TERCEIROS PJ- PAGTO ANTECIPADO']
df_cnpj = df[df['CREDOR'] != '160129']

df_cnpj['CREDOR'].unique()

array(['01590728000264', '05462543000144', '08792763000124',
       '11142525000188', '28811718000187', '31053239000153',
       '33658130000175', '37583132000101', '01590728000850',
       '05872523000141', '06981180000116', '09583781000169',
       '12742245000173', '17281106000103', '20584944000160',
       '27325275000151', '27995686000154', '34028316001509',
       '39319890000134', '41505690000118', '41776201000162',
       '43003732000148', '43433811000199', '44812602000119',
       '45424853000199', '76535764000143', '00139736000145',
       '01255291000121', '01804159000121', '02338743000100',
       '02347121000210', '03157268000120', '03539933000140',
       '04785664000164', '05088041000103', '06022475000165',
       '06131805000150', '06894510000136', '07027329743',
       '07457410000296', '07912053000128', '08605233000120',
       '08703140000138', '09035549000196', '09249668000141',
       '09505477000101', '10463704000154', '10526703000101',
       '10842829000195', '1

In [24]:
df_cnpj[df_cnpj['CREDOR'] == '160129']

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO


In [25]:
ano_atual = str(datetime.now().year)


df_corrente = df[df['ANO'] == ano_atual]
df_corrente

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,01590728000264,MICROTECNICA INFORMATICA LTDA,2023-11-01,1,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905230,MAQUINAS E EQUIPAMENTOS ENERGETICOS,2023NE001319,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,DF,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,3720.60,0.0,3720.60,0.0
1,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,05462543000144,EASYTECH INFORMATICA E SERVICOS LTDA,2023-10-31,2,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905237,EQUIPAMENTOS DE TIC - ATIVOS DE REDE,2023NE001318,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,DF,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,8070.92,0.0,8070.92,0.0
2,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,08792763000124,MEGA BYTE MAGAZINE LTDA,2023-10-31,2,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE001317,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,26270.00,0.0,26270.00,0.0
3,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,11142525000188,CLEBER NASCIMENTO DA ROSA,2023-11-01,1,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905233,"EQUIPAMENTOS PARA AUDIO, VIDEO E FOTO",2023NE001322,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,SC,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,260.00,0.0,260.00,0.0
4,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,28811718000187,BRUNO DIGITAL COMERCIO DE MERCADORIA EM GERAL ...,2023-10-31,2,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE001316,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,79002.00,0.0,79002.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3261,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,65095531000150,LOGOS PAPELARIA LTDA,2023-05-25,161,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905245,EQUIPAMENTOS DE TIC - IMPRESSORAS,2023NE001044,E3PCFSCINFO,INFORMATICA,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,167504,COEX - GESTOR,E3PC,FSC,INFO,0.00,0.0,0.00,1800.0
3262,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,81243735000903,POSITIVO TECNOLOGIA S.A.,2023-02-15,260,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE000261,E3PCFSCINFO,INFORMATICA,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,BA,MG,167504,COEX - GESTOR,E3PC,FSC,INFO,0.00,0.0,0.00,7114.0
3263,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,98488040644,MARIO HENRIQUE CUBBI,2023-05-22,164,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903001,COMBUSTIVEIS E LUBRIFICANTES AUTOMOTIVOS,2023NE001035,E3PCFSCCOLU,COMBUSTIVEIS E LUBRIFICANTES,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,167504,COEX - GESTOR,E3PC,FSC,COLU,0.00,0.0,0.00,2236.2
3264,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,98488040644,MARIO HENRIQUE CUBBI,2023-08-31,63,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903096,MATERIAL DE CONSUMO - PAGTO ANTECIPADO,2023NE001506,E3PCOPFCOLU,COMBUSTIVEL E LUBRIFICANTES,6012,215839,ANO,ORDINARIO,ESCOLA DE 

In [26]:
df['ANO'].unique()

array(['2023', '2022', '2021'], dtype=object)

# Pesquisas no DataFrame

In [27]:
# Criando um DataFrame para testes 
df_teste = df.copy()

## Empenhos com mais de 100 dias

### Empenhos da ND 30

In [28]:
# Empenhos da ND 30 com prazo de entrega superior a 100 dias

df_nd30_100dias = df_teste[(df_teste['A_LIQUIDAR'] != 0) & (df_teste['ND'] == '339030') & (df_teste['DIAS'] > 100) & (df_teste['TIPO_NE'] == 'ORDINARIO') ].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_nd30_100dias.sort_values(by='A_LIQUIDAR', ascending=False).reset_index(drop=True)

#df_nd30_100dias.to_excel('nd30_PA.xlsx')

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,13754000000129,"G4 SERVICO E COMERCIO DE PRODUTOS INDUSTRIAIS,...",2023-06-02,153,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903039,MATERIAL P/ MANUTENCAO DE VEICULOS,2023NE001125,E3PCFSCMABM,MANUTENCAO DE BENS MOVEIS,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,RJ,MG,167504,COEX - GESTOR,E3PC,FSC,MABM,9999.67,0.0,9999.67,0.0
1,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,28259514000185,CLEVER FERREIRA COSTA 01175363693,2023-05-03,183,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903021,MATERIAL DE COPA E COZINHA,2023NE000465,E6SUPLJA5PA,A5PA - PASA,0032,171397,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160504,COEX/GESTOR,E6SU,PLJ,A5PA,3044.75,0.0,3044.75,0.0
2,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,09583781000169,2 CLICK SOLUCOES LTDA,2023-06-06,149,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE000635,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,GO,MG,160073,DGO,I3DA,FUN,ADOM,840.00,0.0,840.00,0.0
3,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,02852112698,LUCIANO ALVES DE PAULA,2023-06-13,142,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903096,MATERIAL DE CONSUMO - PAGTO ANTECIPADO,2023NE001191,E3PCOPFCOLU,COMBUSTIVEL E LUBRIFICANTES,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MS,MG,167504,COEX - GESTOR,E3PC,OPF,COLU,5.22,0.0,5.22,0.0


### Empenhos ND 52

In [29]:
df_nd52_100dias = df_teste[(df_teste['A_LIQUIDAR'] != 0) & (df_teste['ND'] == '449052') & (df_teste['DIAS'] > 50) ].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_nd52_100dias.reset_index(drop=True)

#df_nd52_100dias.to_excel('nd52_PA.xlsx')

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,31669124000198,LS REFRIGERACAO LTDA,2023-04-18,198,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905228,MAQUINAS E EQUIPAMENTOS DE NATUREZA INDUSTRIAL,2023NE000392,E6SUPLJA5PA,A5PA - PASA,0032,171397,ANO,ORDINARIO,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",SC,MG,160504,COEX/GESTOR,E6SU,PLJ,A5PA,8837.42,0.0,8837.42,0.0
1,160129,ESA,21A0,APRESTAMENTO DAS FORCAS,2023,48241204000140,EQUIPAT - EQUIPAMENTOS E MATERIAIS ELETRICOS E...,2023-06-16,139,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905238,"MAQ., FERRAMENTAS E UTENSILIOS DE OFICINA",2023NE000668,E5MMPDRDEGE,DESCENTRALIZACAO RECURSOS MOTOMECANIZACAO,6012,171470,ANO,ORDINARIO,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",RS,MG,160504,COEX/GESTOR,E5MM,PDR,DEGE,995.00,0.0,995.00,0.0
2,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,28320324000126,FREDERICO DELGADO DE ALMEIDA,2023-06-19,136,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905224,"EQUIPAMENTO DE PROTECAO, SEGURANCA E SOCORRO",2023NE000672,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,1379.10,0.0,1379.10,0.0
3,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,81243735001977,POSITIVO TECNOLOGIA S.A.,2023-06-28,127,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE000781,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,AM,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,21342.00,0.0,21342.00,0.0
4,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,81243735000903,POSITIVO TECNOLOGIA S.A.,2023-07-24,101,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE000889,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,BA,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,11454.00,0.0,11454.00,0.0
5,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,10902067000175,ESCRIBLU COMERCIO DE MOVEIS LTDA,2023-07-31,94,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905242,MOBILIARIO EM GERAL,2023NE000938,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SC,MG,160073,DGO,I3DA,FUN,ADOM,3384.95,0.0,3384.95,0.0
6,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,10902067000175,ESCRIBLU COMERCIO DE MOVEIS LTDA,2023-08-02,92,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905242,MOBILIARIO EM GERAL,2023NE000951,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SC,MG,160073,DGO,I3DA,FUN,ADOM,2030.97,0.0,2030.97,0.0
7,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,10902067000175,ESCRIBLU COMERCIO DE MOVEIS LTDA,2023-08-02,92,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905242,MOBILIARIO EM GERAL,2023NE000952,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SC,MG,160073,DGO,I3DA,FUN,ADOM,4063.56,0.0,4063.56,0.0
8,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,10902067000175,ESCRIBLU COMERCIO DE MOVEIS LTDA,2023-08-02,92,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905242,MOBILIARIO EM GERAL,2023NE001377,D5SAFUSASOC,ASSISTENCIA SOCIAL,0032,215846,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,SC,MG,167505,DGP,D5SA,FUS,ASOC,2707.96,0.0,2707.96,0.0
9,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,10902067000175,ESCRIBLU COMERCIO DE MOVEIS LTDA,2023-08-02,92,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905242,MOBILIARIO EM GERAL,2023

### Consolidação ND 30 e ND 39

In [38]:
df_foco.columns

Index(['UG', 'SIGLA UG', 'ACAO', 'NOME_ACAO', 'ANO', 'CREDOR', 'NOME_CREDOR',
       'DATA', 'DIAS', 'ICFEX', 'NOME_ICFEX', 'ND', 'NOME_ND', 'NDSI',
       'NOME_NDSI', 'NE', 'PI', 'NOME_PI', 'PROGRAMA', 'PTRES', 'TIPO',
       'TIPO_NE', 'LOCAL', 'UF_CREDOR', 'UF_UG', 'UGR', 'NOME_UGR', 'RADPI4',
       'MEIPI3', 'SUFPI4', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR',
       'PAGO'],
      dtype='object')

In [44]:
cols = ['UG', 'ANO', 'CREDOR', 'NOME_CREDOR',
       'DATA', 'DIAS', 'ND', 'NDSI', 'NOME_NDSI', 
       'NE', 'PI', 'NOME_PI', 'UGR', 'NOME_UGR', 
       'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR','PAGO']
df_foco.loc[:, cols]

,UG,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,UGR,NOME_UGR,A_LIQUIDAR,LIQUIDADO_A_PAGAR,PAGO
0,160129,2023,10653955000100,I&P AGROCOMERCIAL LTDA,2023-02-13,262,339030,33903007,GENEROS DE ALIMENTACAO,2023NE000114,E6SUPLJA1QR,A1QR - QUANTITATIVO DE RANCHO,160504,COEX/GESTOR,2670.68,0.0,94485.32
1,160129,2023,19791896006486,SUPERGASBRAS ENERGIA LTDA,2023-02-14,261,339030,33903004,GAS E OUTROS MATERIAIS ENGARRAFADOS,2023NE000120,E6SUPLJA1QR,A1QR - QUANTITATIVO DE RANCHO,160504,COEX/GESTOR,8728.03,0.0,171196.20
2,160129,2023,20702528000110,CARLOS GABRIEL DOS REIS,2023-02-15,260,339030,33903007,GENEROS DE ALIMENTACAO,2023NE000158,E6SUPLJESCO,ESCO - QUANTITATIVO DE RANCHO,160504,COEX/GESTOR,12930.36,0.0,123555.47
3,160129,2023,31669124000198,LS REFRIGERACAO LTDA,2023-04-18,198,449052,44905228,MAQUINAS E EQUIPAMENTOS DE NATUREZA INDUSTRIAL,2023NE000392,E6SUPLJA5PA,A5PA - PASA,160504,COEX/GESTOR,8837.42,0.0,0.00
4,160129,2023,28259514000185,CLEVER FERREIRA COSTA 01175363693,2023-05-03,183,339030,33903021,MATERIAL DE COPA E COZINHA,2023NE000465,E6SUPLJA5PA,A5PA - PASA,160504,COEX/GESTOR,3044.75,0.0,0.00
5,160129,2023,19791896006486,SUPERGASBRAS ENERGIA LTDA,2023-05-29,157,339030,33903004,GAS E OUTROS MATERIAIS ENGARRAFADOS,2023NE000559,C4ENMILCAPE,CAPACITACAO DE PESSOAL,160503,DECEX/GESTOR,1850.42,0.0,0.00
6,167129,2023,13754000000129,"G4 SERVICO E COMERCIO DE PRODUTOS INDUSTRIAIS,...",2023-06-02,153,339030,33903039,MATERIAL P/ MANUTENCAO DE VEICULOS,2023NE001125,E3PCFSCMABM,MANUTENCAO DE BENS MOVEIS,167504,COEX - GESTOR,9999.67,0.0,0.00
7,160129,2023,09583781000169,2 CLICK SOLUCOES LTDA,2023-06-06,149,339030,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE000635,I3DAFUNADOM,ADMINISTRACAO DE OM,160073,DGO,840.00,0.0,0.00
8,160129,2023,43928711000133,GARDEN MATERIAIS DE CONSTRUCAO LTDA,2023-06-07,148,339030,33903024,MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES,2023NE000647,E6RVSOLOUT4,REMONTA E VETERINÁRIA,160504,COEX/GESTOR,955.00,0.0,3830.00
9,167129,2023,02852112698,LUCIANO ALVES DE PAULA,2023-06-13,142,339030,33903096,MATERIAL DE CONSUMO - PAGTO ANTECIPADO,2023NE001191,E3PCOPFCOLU,COMBUSTIVEL E LUBRIFICANTES,167504,COEX - GESTOR,5.22,0.0,0.00


In [37]:
nd = ['449052', '339030' ]

df_foco = df_teste.loc[df_teste['ND'].isin(nd) & (df_teste['DIAS'] > 60) & (df_teste['A_LIQUIDAR'] != 0), :].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_foco.reset_index(drop=True)



,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,10653955000100,I&P AGROCOMERCIAL LTDA,2023-02-13,262,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903007,GENEROS DE ALIMENTACAO,2023NE000114,E6SUPLJA1QR,A1QR - QUANTITATIVO DE RANCHO,0032,171397,ANO,GLOBAL,"AV. SETE DE SETEMBRO, 628, CENTRO, TRES CORAÇÕ...",MG,MG,160504,COEX/GESTOR,E6SU,PLJ,A1QR,2670.68,0.0,2670.68,94485.32
1,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,19791896006486,SUPERGASBRAS ENERGIA LTDA,2023-02-14,261,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903004,GAS E OUTROS MATERIAIS ENGARRAFADOS,2023NE000120,E6SUPLJA1QR,A1QR - QUANTITATIVO DE RANCHO,0032,171397,ANO,ESTIMATIVO,"AV. SETE DE SETEMBRO, 628, CENTRO, TRES CORAÇÕ...",SP,MG,160504,COEX/GESTOR,E6SU,PLJ,A1QR,8728.03,0.0,8728.03,171196.20
2,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,20702528000110,CARLOS GABRIEL DOS REIS,2023-02-15,260,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903007,GENEROS DE ALIMENTACAO,2023NE000158,E6SUPLJESCO,ESCO - QUANTITATIVO DE RANCHO,0032,171397,ANO,GLOBAL,"AV. SETE DE SETEMBRO, 628, CENTRO, TRES CORAÇÕ...",MG,MG,160504,COEX/GESTOR,E6SU,PLJ,ESCO,12930.36,0.0,12930.36,123555.47
3,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,31669124000198,LS REFRIGERACAO LTDA,2023-04-18,198,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905228,MAQUINAS E EQUIPAMENTOS DE NATUREZA INDUSTRIAL,2023NE000392,E6SUPLJA5PA,A5PA - PASA,0032,171397,ANO,ORDINARIO,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",SC,MG,160504,COEX/GESTOR,E6SU,PLJ,A5PA,8837.42,0.0,8837.42,0.00
4,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,28259514000185,CLEVER FERREIRA COSTA 01175363693,2023-05-03,183,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903021,MATERIAL DE COPA E COZINHA,2023NE000465,E6SUPLJA5PA,A5PA - PASA,0032,171397,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160504,COEX/GESTOR,E6SU,PLJ,A5PA,3044.75,0.0,3044.75,0.00
5,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,19791896006486,SUPERGASBRAS ENERGIA LTDA,2023-05-29,157,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903004,GAS E OUTROS MATERIAIS ENGARRAFADOS,2023NE000559,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,1850.42,0.0,1850.42,0.00
6,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,13754000000129,"G4 SERVICO E COMERCIO DE PRODUTOS INDUSTRIAIS,...",2023-06-02,153,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903039,MATERIAL P/ MANUTENCAO DE VEICULOS,2023NE001125,E3PCFSCMABM,MANUTENCAO DE BENS MOVEIS,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,RJ,MG,167504,COEX - GESTOR,E3PC,FSC,MABM,9999.67,0.0,9999.67,0.00
7,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,09583781000169,2 CLICK SOLUCOES LTDA,2023-06-06,149,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE000635,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,GO,MG,160073,DGO,I3DA,FUN,ADOM,840.00,0.0,840.00,0.00
8,160129,ESA,21A0,APRESTAMENTO DAS FORCAS,2023,43928711000133,GARDEN MATERIAIS DE CONSTRUCAO LTDA,2023-06-07,148,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903024,MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES,2023NE000647,E6RVSOLOUT4,REMONTA E VETERINÁRIA,6012,193910,ANO,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160504,COEX/GESTOR,E6RV,SOL,OUT4,955.00,0.0,955.00,3830.00
9,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,02852112698,LUCIANO ALVES DE PAULA,2023-06-13,142,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903096,MATERIAL DE CONSUMO - PAGTO ANTECIPADO,2023NE001191,E3PCOPFCOLU,COM

### Recursos Liquidados a pagar

In [31]:
df_liqui_a_pagar =df_teste[ df_teste['LIQUIDADO_A_PAGAR'] != 0 ]

df_liqui_a_pagar.sort_values(by='LIQUIDADO_A_PAGAR', ascending=False).reset_index(drop=True).head(2)

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,08906031000118,AMAZING METALURGICA LTDA,2023-08-28,66,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905242,MOBILIARIO EM GERAL,2023NE001428,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,0.0,139000.0,139000.0,0.0
1,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,26401571000121,TRES PHARMA DISTRIBUIDORA E SERVICOS LTDA,2023-10-05,28,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903009,MATERIAL FARMACOLOGICO,2023NE001633,D8SAFUSMTOS,MEDICAMENTOS DE ALTO CUSTO,0032,215845,ANO,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,167505,DGP,D8SA,FUS,MTOS,0.0,48731.4,48731.4,0.0


In [32]:
total_liqui_a_pagar = df_liqui_a_pagar['LIQUIDADO_A_PAGAR'].sum()
locale.currency(total_liqui_a_pagar, grouping=True)

'R$ 393.153,53'

In [33]:
df_aux = df_liqui_a_pagar.groupby(by='NOME_NDSI')['LIQUIDADO_A_PAGAR'].sum().sort_values( ascending=False).reset_index()

fig = px.bar(df_aux,
             x = 'NOME_NDSI',
             y = 'LIQUIDADO_A_PAGAR',
             log_y=True,
             height=900,
             title='Saldos liquidados a pagar por NDSI',
             labels={'NOME_NDSI':'Descrição do Subitem', 'LIQUIDADO_A_PAGAR':'Liquidado a Pagar'},
             text_auto='.3s',
             color='LIQUIDADO_A_PAGAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [34]:
df_teste[df_teste['NE'] == '2023NE000392']

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
351,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,31669124000198,LS REFRIGERACAO LTDA,2023-04-18,198,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905228,MAQUINAS E EQUIPAMENTOS DE NATUREZA INDUSTRIAL,2023NE000392,E6SUPLJA5PA,A5PA - PASA,0032,171397,ANO,ORDINARIO,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",SC,MG,160504,COEX/GESTOR,E6SU,PLJ,A5PA,8837.42,0.0,8837.42,0.00
3063,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,46940235787,DAVILAN DE SOUZA PORCARO,2023-02-27,248,160533,4º CGCFEX,339036,OUTROS SERVICOS DE TERCEIROS - PESSOA FISICA,33903630,SERVICOS MEDICOS E ODONTOLOGICOS,2023NE000392,D8SAFUSPRSA,PROFISSIONAL DE SAUDE AUTONOMO,0032,215845,ANO,ESTIMATIVO,CLINICA/CONSULTÓRIO/HOSPITAL,MG,MG,167505,DGP,D8SA,FUS,PRSA,0.00,0.0,0.00,648.27


In [35]:
df_teste

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,01590728000264,MICROTECNICA INFORMATICA LTDA,2023-11-01,1,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905230,MAQUINAS E EQUIPAMENTOS ENERGETICOS,2023NE001319,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,DF,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,3720.60,0.0,3720.60,0.0
1,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,05462543000144,EASYTECH INFORMATICA E SERVICOS LTDA,2023-10-31,2,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905237,EQUIPAMENTOS DE TIC - ATIVOS DE REDE,2023NE001318,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,DF,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,8070.92,0.0,8070.92,0.0
2,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,08792763000124,MEGA BYTE MAGAZINE LTDA,2023-10-31,2,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE001317,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,26270.00,0.0,26270.00,0.0
3,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,11142525000188,CLEBER NASCIMENTO DA ROSA,2023-11-01,1,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905233,"EQUIPAMENTOS PARA AUDIO, VIDEO E FOTO",2023NE001322,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,SC,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,260.00,0.0,260.00,0.0
4,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,28811718000187,BRUNO DIGITAL COMERCIO DE MERCADORIA EM GERAL ...,2023-10-31,2,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE001316,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,79002.00,0.0,79002.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3261,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,65095531000150,LOGOS PAPELARIA LTDA,2023-05-25,161,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905245,EQUIPAMENTOS DE TIC - IMPRESSORAS,2023NE001044,E3PCFSCINFO,INFORMATICA,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,167504,COEX - GESTOR,E3PC,FSC,INFO,0.00,0.0,0.00,1800.0
3262,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,81243735000903,POSITIVO TECNOLOGIA S.A.,2023-02-15,260,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE000261,E3PCFSCINFO,INFORMATICA,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,BA,MG,167504,COEX - GESTOR,E3PC,FSC,INFO,0.00,0.0,0.00,7114.0
3263,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,98488040644,MARIO HENRIQUE CUBBI,2023-05-22,164,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903001,COMBUSTIVEIS E LUBRIFICANTES AUTOMOTIVOS,2023NE001035,E3PCFSCCOLU,COMBUSTIVEIS E LUBRIFICANTES,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,167504,COEX - GESTOR,E3PC,FSC,COLU,0.00,0.0,0.00,2236.2
3264,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,98488040644,MARIO HENRIQUE CUBBI,2023-08-31,63,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903096,MATERIAL DE CONSUMO - PAGTO ANTECIPADO,2023NE001506,E3PCOPFCOLU,COMBUSTIVEL E LUBRIFICANTES,6012,215839,ANO,ORDINARIO,ESCOLA DE 

In [36]:
df_restos = df_teste[(df_teste['ANO'] == '2022') & (df_teste['A_LIQUIDAR'] !=  0)].sort_values(by='A_LIQUIDAR', ascending=False)

df_restos

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
954,160129,ESA,219D,ADEQUACAO DE ATIVOS DE INFRAESTRUTURA,2022,17289559000186,FENARA & FENARA - ENGENHARIA E ADMINISTRACAO LTDA,2022-12-14,323,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903916,MANUTENCAO E CONSERV. DE BENS IMOVEIS,2022NE002483,B4OMOBMAQUA,AQUARTELAMENTO,6012,171459,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160502,DEC-GESTOR,B4OM,OBM,AQUA,263051.08,0.0,263051.08,171948.92
953,160129,ESA,219D,ADEQUACAO DE ATIVOS DE INFRAESTRUTURA,2022,17289559000186,FENARA & FENARA - ENGENHARIA E ADMINISTRACAO LTDA,2022-12-14,323,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903916,MANUTENCAO E CONSERV. DE BENS IMOVEIS,2022NE002482,B4OMOBMAQUA,AQUARTELAMENTO,6012,171459,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160502,DEC-GESTOR,B4OM,OBM,AQUA,155796.97,0.0,155796.97,204469.27
955,160129,ESA,219D,ADEQUACAO DE ATIVOS DE INFRAESTRUTURA,2022,17289559000186,FENARA & FENARA - ENGENHARIA E ADMINISTRACAO LTDA,2022-12-28,309,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903916,MANUTENCAO E CONSERV. DE BENS IMOVEIS,2022NE002701,B4OMOBMAQUA,AQUARTELAMENTO,6012,171459,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,SP,MG,160502,DEC-GESTOR,B4OM,OBM,AQUA,51192.83,0.0,51192.83,98807.17
1150,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2022,17289559000186,FENARA & FENARA - ENGENHARIA E ADMINISTRACAO LTDA,2022-12-28,309,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903916,MANUTENCAO E CONSERV. DE BENS IMOVEIS,2022NE002702,C1ENMILOUTR,OUTROS,6012,171423,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,SP,MG,160503,DECEX/GESTOR,C1EN,MIL,OUTR,12733.76,0.0,12733.76,0.00
1157,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2022,26669742000106,J. T. CATALDO CASTRO CRIATIVA DIGITAL,2022-11-30,337,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903963,SERVICOS GRAFICOS E EDITORIAIS,2022NE002421,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,RPNP,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,RJ,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,5758.75,0.0,5758.75,0.00
1167,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2022,38680138000151,GESMAQ COMERCIO E SERVICOS LTDA,2022-10-06,392,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903917,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,2022NE001085,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,5543.35,0.0,5543.35,27716.65
1168,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2022,38680138000151,GESMAQ COMERCIO E SERVICOS LTDA,2022-10-06,392,160533,4º CGCFEX,339040,SERVIÇOS DE TECNOLOGIA DA INFORMAÇÃO E COMUNIC...,33904012,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,2022NE001086,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,3466.70,0.0,3466.70,17333.30
1134,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2022,05691235000190,COPYUAI LOCACAO DE EQUIPAMENTOS LTDA,2022-10-06,392,160533,4º CGCFEX,339040,SERVIÇOS DE TECNOLOGIA DA INFORMAÇÃO E COMUNIC...,33904012,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,2022NE001087,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,2397.00,0.0,2397.00,7191.00
1419,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,26669742000106,J. T. CATALDO CASTRO CRIATIVA DIGITAL,2022-11-30,337,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903963,SERVICOS GRAFICOS E EDITORIAIS,2022NE001843,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,RPNP,OR